In [1]:
%%time
# Import libraries and load monthly stock price data

import numpy as np
import pandas as pd
import math
rf= 0.000195179701757642


##Mean portfolio return = Mean Return * Fractions of Total Capital (Chromosome). 
# Risk-free rate = 0.05 (assumption) 
# Standard deviation of portfolio return = (chromosome * Standard deviation)**2 + Covariance * Respective weights in chromosome.
def mean_portfolio_return(child):
    return np.sum(np.multiply(child,mean_hist_return))
def var_portfolio_return(child):
    part_1 = np.sum(np.multiply(child,sd_hist_return)**2)
    temp_lst=[]
    for i in range(50):
        for j in range(50):
            temp=cov_hist_return.iloc[i][j] * child[i] * child[j]
            temp_lst.append(temp)
    part_2=np.sum(temp_lst)
    return part_1+part_2

# Chromosome: Set of genes i.e. fractions of total capital assigned to each asset
def chromosome(n):
    ''' Generates set of random numbers whose sum is equal to 1
        Input: Number of stocks.
        Output: Array of random numbers'''
    ch = np.random.rand(n)
    return ch/sum(ch)

## Fitness function : The Sharpe ratio, S = (µ − r)/σ
## Maximize return, minimize risk
def fitness_fuction(child):
    ''' This will return the Sharpe ratio for a particular portfolio.
        Input: A child/chromosome (1D Array)
        Output: Sharpe Ratio value (Scalar)'''
    return (mean_portfolio_return(child)-rf)/np.sqrt(var_portfolio_return(child))

def Select_elite_population(population, frac=0.3):
    ''' Select elite population from the total population based on fitness function values.
        Input: Population and fraction of population to be considered as elite.
        Output: Elite population.'''
    population = sorted(population,key = lambda x: fitness_fuction(x),reverse=True)
    percentage_elite_idx = int(np.floor(len(population)* frac))
    return population[:percentage_elite_idx]

def mutation(parent):
    ''' Randomy choosen elements of a chromosome are swapped
        Input: Parent
        Output: Offspring (1D Array)'''
    child=parent.copy()
    n=np.random.choice(range(50),2)
    while (n[0]==n[1]):
        n=np.random.choice(range(50),2)
    child[n[0]],child[n[1]]=child[n[1]],child[n[0]]
    return child

def Heuristic_crossover(parent1,parent2):
    ''' The oﬀsprings are created according to the equation:
            Off_spring A = Best Parent  + β ∗ ( Best Parent − Worst Parent)
            Off_spring B = Worst Parent - β ∗ ( Best Parent − Worst Parent)
                Where β is a random number between 0 and 1.
        Input: 2 Parents
        Output: 2 Children (1d Array)'''
    ff1=fitness_fuction(parent1)
    ff2=fitness_fuction(parent2)
    diff=parent1 - parent2
    beta=np.random.rand()
    if ff1>ff2:
        child1=parent1 + beta * diff
        child2=parent2 - beta * diff
    else:
        child2=parent1 + beta * diff
        child1=parent2 - beta * diff
    return child1,child2

def Arithmetic_crossover(parent1,parent2):
    ''' The oﬀsprings are created according to the equation:
            Off spring A = α ∗ Parent1 + (1 −α) ∗ Parent2
            Off spring B = (1 −α) ∗ Parent1 + α ∗ Parent2
            
                Where α is a random number between 0 and 1.
        Input: 2 Parents
        Output: 2 Children (1d Array)'''
    alpha = np.random.rand()
    child1 = alpha * parent1 + (1-alpha) * parent2
    child2 = (1-alpha) * parent1 + alpha * parent2
    return child1,child2

def next_generation(pop_size,elite,crossover=Arithmetic_crossover):
    ''' Generates new population from elite population with mutation probability as 0.4 and crossover as 0.6. 
        Over the final stages, mutation probability is decreased to 0.1.
        Input: Population Size and elite population.
        Output: Next generation population (2D Array).'''
    new_population=[]
    elite_range=range(len(elite))
    #print(elite_range)
    while len(new_population) < pop_size:
        if len(new_population) > 2*pop_size/3: # In the final stages mutation frequency is decreased.
            mutate_or_crossover = np.random.choice([0, 1], p=[0.6, 0.4])
        else:
            mutate_or_crossover = np.random.choice([0, 1], p=[0.4, 0.6])
        #print(mutate_or_crossover)
        if mutate_or_crossover:
            indx=np.random.choice(elite_range)
            new_population.append(mutation(elite[indx]))
            #print('Mutation!')
        else:
            p1_idx,p2_idx=np.random.choice(elite_range,2)
            c1,c2=crossover(elite[p1_idx],elite[p2_idx])
            chk=0
            for gene in range(50):
                if c1[gene]<0:
                    chk+=1
                else:
                    chk+=0
            if chk>0:
                p1_idx,p2_idx=np.random.choice(elite_range,2)
                c1,c2=crossover(elite[p1_idx],elite[p2_idx])
            
            new_population.extend([c1,c2])
            #print('Crossover!')    
    return new_population


hist_stock_returns = pd.read_csv('Nifty50returnsJuly2018-Jun2021.csv')
hist_stock_returns['date'] = pd.to_datetime(hist_stock_returns['date'], format='%d-%m-%Y')
hist_stock_returns.set_index('date',inplace=True)

mean_hist_return=hist_stock_returns.mean()
#print(mean_hist_return)
sd_hist_return=hist_stock_returns.std()
#print(sd_hist_return)
cov_hist_return=hist_stock_returns.cov()
#print(cov_hist_return)
for i in range(50):
    cov_hist_return.iloc[i][i]=0
#print(cov_hist_return)

    
## Determine the Optimum portfolio using Arithmetic Crossover  

n=50 # Number of stocks = 6
pop_size=250 # initial population = 100

# Initial population
population = np.array([chromosome(n) for _ in range(pop_size)])

# Get initial elite population
elite = Select_elite_population(population)

iteration=0 
Expected_returns=10
Expected_risk=15

plotData = {
    "expReturns": [],
    "expRisk": [],
    "SharpeRatio": [] 
}

while (Expected_returns < 30 or Expected_risk > 10) and iteration <= 300:
        
    population = next_generation(250,elite)
    elite = Select_elite_population(population)
    
    Expected_returns=(((1+mean_portfolio_return(elite[0]))**250)-1)*100
    Expected_risk=math.sqrt(var_portfolio_return(elite[0]))*math.sqrt(250)*100
    SR = (Expected_returns-5)/Expected_risk
    
    print('Iteration:',iteration)    
    print('Expected returns of {} with risk of {} and SR = {}\n'.format(round(Expected_returns,ndigits=3),round(Expected_risk,ndigits=3),round(SR,ndigits=3)))
    plotData['expReturns'].extend([Expected_returns])
    plotData['expRisk'].extend([Expected_risk])
    plotData['SharpeRatio'].extend([SR])
    
    iteration+=1

print('\n\n=====================================================\nPortfolio of stocks after all the iterations:\n')
[print(hist_stock_returns.columns[i],':',elite[0][i]) for i in list(range(50))]

print('\nExpected returns of {} with risk of {} and Sharpe Ratio of {}\n'.format(round(Expected_returns,ndigits=3),round(Expected_risk,ndigits=3),round(SR,ndigits=3)))

companies = []
wts = []
for i in list(range(50)):
    companies = companies + [hist_stock_returns.columns[i]]
    wts = wts + [elite[0][i]]

wtDf = pd.DataFrame(columns=['Company','Weight'])
wtDf.Company = companies
wtDf.Weight = wts
wtDf.to_csv('OptimizedWeightsJul2021-Dec2021.csv',index=False)

Iteration: 0
Expected returns of 24.174 with risk of 21.554 and SR = 0.89

Iteration: 1
Expected returns of 23.721 with risk of 20.805 and SR = 0.9

Iteration: 2
Expected returns of 24.654 with risk of 21.69 and SR = 0.906

Iteration: 3
Expected returns of 25.703 with risk of 22.01 and SR = 0.941

Iteration: 4
Expected returns of 25.699 with risk of 22.008 and SR = 0.941

Iteration: 5
Expected returns of 25.705 with risk of 22.015 and SR = 0.94

Iteration: 6
Expected returns of 25.483 with risk of 21.818 and SR = 0.939

Iteration: 7
Expected returns of 26.041 with risk of 21.863 and SR = 0.962

Iteration: 8
Expected returns of 26.213 with risk of 21.526 and SR = 0.985

Iteration: 9
Expected returns of 26.462 with risk of 21.67 and SR = 0.99

Iteration: 10
Expected returns of 26.328 with risk of 21.548 and SR = 0.99

Iteration: 11
Expected returns of 26.217 with risk of 21.523 and SR = 0.986

Iteration: 12
Expected returns of 26.483 with risk of 21.338 and SR = 1.007

Iteration: 13
Expe

Iteration: 107
Expected returns of 28.884 with risk of 21.145 and SR = 1.13

Iteration: 108
Expected returns of 28.832 with risk of 21.103 and SR = 1.129

Iteration: 109
Expected returns of 28.834 with risk of 21.104 and SR = 1.129

Iteration: 110
Expected returns of 28.843 with risk of 21.112 and SR = 1.129

Iteration: 111
Expected returns of 28.857 with risk of 21.118 and SR = 1.13

Iteration: 112
Expected returns of 28.822 with risk of 21.085 and SR = 1.13

Iteration: 113
Expected returns of 28.832 with risk of 21.096 and SR = 1.13

Iteration: 114
Expected returns of 28.864 with risk of 21.121 and SR = 1.13

Iteration: 115
Expected returns of 28.866 with risk of 21.123 and SR = 1.13

Iteration: 116
Expected returns of 28.876 with risk of 21.127 and SR = 1.13

Iteration: 117
Expected returns of 28.874 with risk of 21.127 and SR = 1.13

Iteration: 118
Expected returns of 28.874 with risk of 21.127 and SR = 1.13

Iteration: 119
Expected returns of 28.874 with risk of 21.127 and SR = 1.

Iteration: 214
Expected returns of 28.811 with risk of 21.059 and SR = 1.131

Iteration: 215
Expected returns of 28.821 with risk of 21.067 and SR = 1.131

Iteration: 216
Expected returns of 28.802 with risk of 21.052 and SR = 1.131

Iteration: 217
Expected returns of 28.815 with risk of 21.063 and SR = 1.131

Iteration: 218
Expected returns of 28.815 with risk of 21.062 and SR = 1.131

Iteration: 219
Expected returns of 28.816 with risk of 21.064 and SR = 1.131

Iteration: 220
Expected returns of 28.816 with risk of 21.063 and SR = 1.131

Iteration: 221
Expected returns of 28.813 with risk of 21.06 and SR = 1.131

Iteration: 222
Expected returns of 28.813 with risk of 21.061 and SR = 1.131

Iteration: 223
Expected returns of 28.815 with risk of 21.062 and SR = 1.131

Iteration: 224
Expected returns of 28.815 with risk of 21.062 and SR = 1.131

Iteration: 225
Expected returns of 28.814 with risk of 21.061 and SR = 1.131

Iteration: 226
Expected returns of 28.814 with risk of 21.062 and

Wall time: 15h 43min 22s


In [2]:
pd.DataFrame(plotData).to_csv('C2Results.csv')